## benchmark (Polyglot)

In [ ]:
//// test

open testing

In [ ]:
inl gc_collect () =
    run_target function
        | Fsharp _ => fun () => $'System.GC.Collect' ()
        | _ => fun () => ()
        
inl printfn x =
    console.write_line x

nominal stopwatch = $'System.Diagnostics.Stopwatch'

inl stopwatch () : stopwatch =
    $'`stopwatch ' ()

inl stopwatch_elapsed_milliseconds (stopwatch : stopwatch) : i64 =
    $'!stopwatch.ElapsedMilliseconds'

inl stopwatch_start (stopwatch : stopwatch) : () =
    $'!stopwatch.Start' ()

## test_case_result

In [ ]:
type test_case_result =
    {
        Input : string
        Expected : string
        Result : string
        TimeList : a u64 i64
    }

## run

In [ ]:
inl run forall input expected.
    count
    (solutions : list (string * (input -> expected)))
    ((input, expected) : (input * expected))
    : test_case_result
    =
    inl input_str = input |> sm'.format_debug

    printfn ""
    printfn ($'$"Solution: {!input_str}  "' : string)

    inl performance_invoke (fn : () -> expected) =
        gc_collect ()
        inl stopwatch = stopwatch ()
        stopwatch |> stopwatch_start
        inl time1 = stopwatch |> stopwatch_elapsed_milliseconds
        inl result : expected =
            am'.init_series 0 count 1i32
            |> am'.parallel_map fun _n => fn ()
            |> am'.last
        inl time2 = (stopwatch |> stopwatch_elapsed_milliseconds) - time1
        result, time2

    inl results_with_time : a u64 _ =
        solutions
        |> listm'.indexed
        |> listm.toArray
        |> am.map fun ((i : i32), (test_name, solution)) =>
            inl result, time = performance_invoke fun () => solution input
            printfn ($'$"Test case {!i + 1}. {!test_name}. Time: {!time}  "' : string)
            result, time

    match results_with_time |> am.map fst with
    | array when length array <= 1 => ()
    | array when array |> am.forall' ((=) (index array 0)) => ()
    | results => failwith ($'$"Challenge error: {!results}"' : string)

    {
        Input = input_str
        Expected = expected |> sm'.format_debug
        Result = results_with_time |> am.map fst |> fun array => index array 0 |> sm'.format_debug
        TimeList = results_with_time |> am.map snd
    }

## run_all

In [ ]:
inl run_all forall input expected.
    test_name
    count
    (solutions : list (string * (input -> expected)))
    test_cases
    =
    printfn ""
    printfn ""
    printfn ($'$"Test: {!test_name}"' : string)
    test_cases
    |> listm.toArray
    |> am.map (run count solutions)

## sort_result_list

In [ ]:
inl sort_result_list results =
    inl table =
        inl rows =
            results
            |> am.map fun (result : test_case_result) =>
                inl best =
                    result.TimeList
                    |> am'.indexed
                    |> am.map fun (i, time) =>
                        i + 1i64, time
                    |> am'.sort_by snd
                    |> fun array => index array 0i32
                    |> sm'.format_debug
                inl row =
                    [
                        result.Input
                        result.Expected
                        result.Result
                        best
                    ]
                inl color : option console.console_color =
                    open console
                    match result.Expected = result.Result with
                    | true => Some $'`console_color.DarkGreen'
                    | false => Some $'`console_color.DarkRed'
                row, color

        inl header =
            [
                [
                    "Input"
                    "Expected"
                    "Result"
                    "Best"
                ]
                [
                    "---"
                    "---"
                    "---"
                    "---"
                ]
            ]
            |> listm.map fun row => row, None
            |> listm.toArray
        rows |> am.append header

    inl formattedTable =
        inl lengthMap : mapm.map i32 i64 =
            table
            |> am.map (fst >> listm.toArray)
            |> am'.transpose
            |> am.map fun column =>
                column
                |> am.map sm.length
                |> am'.sort_descending
                |> am'.try_item 0i32
                |> optionm'.default_value 0i64
            |> am'.indexed
            |> fun (x : a i32 _) => x
            |> mapm.of_array
        table
        |> am.map fun (row, color) =>
            inl newRow =
                row
                |> listm'.indexed
                |> listm.map fun (i, cell) =>
                    cell |> sm'.pad_right (lengthMap |> mapm.item i |> conv) ' '
                |> listm.toArray
            newRow, color

    printfn ""
    formattedTable
    |> am.iter fun ((row : a i32 string), color) =>
        match color with
        | Some color => color |> console.set_foreground_color
        | None => console.reset_color ()

        printfn (row |> sm'.join' "\t| ")

        console.reset_color ()

    inl averages : a u64 _ =
        results
        |> am.map fun result =>
            result.TimeList
            |> am.map ($'float' : i64 -> f64)
        |> am'.transpose
        |> am.map am'.average
        |> am.map ($'int64' : f64 -> i64)
        |> am'.indexed

    printfn ""
    printfn "Average Ranking  "
    averages
    |> am'.sort_by snd
    |> am.iter fun ((i : i32), avg) =>
        printfn ($'$"Test case %d{!i + 1}. Average Time: %A{!avg}  "' : string)

In [ ]:
//// test

inl is_fast () =
    false

## empty2Tests

Test: Empty2

Solution: (a, a)  
Test case 1. A. Time: 59L

Solution: (a, a)  
Test case 1. A. Time: 53L

Input   | Expected        | Result  | Best
---     | ---             | ---     | ---
(a, a)  | a               | a       | (1, 59)
(a, a)  | a               | a       | (1, 53)

Averages  
Test case 1. Average Time: 56L

Ranking  
Test case 1. Average Time: 56L

In [ ]:
//// test

inl get_solutions () =
    [
        "A",
        fun (a, _b) =>
            a

        "B",
        fun (_a, b) =>
            b
    ]

inl rec empty_2_tests () =
    inl test_cases = [
        ("a", "a"), "a"
        ("b", "b"), "b"
    ]

    inl solutions = get_solutions ()

    // inl is_fast () = true

    inl count =
        if is_fast ()
        then 1000i32
        else 2000000i32

    run_all (nameof empty_2_tests) count solutions test_cases
    |> sort_result_list

empty_2_tests ()



Test: v0

Solution: struct ("a", "a")  
Test case 1. A. Time: 390  
Test case 2. B. Time: 334  

Solution: struct ("b", "b")  
Test case 1. A. Time: 291  
Test case 2. B. Time: 316  

Input            	| Expected	| Result	| Best             
---              	| ---     	| ---   	| ---              
struct ("a", "a")	| "a"     	| "a"   	| struct (2L, 334L)
struct ("b", "b")	| "b"     	| "b"   	| struct (1L, 291L)

Average Ranking  
Test case 2. Average Time: 325L  
Test case 1. Average Time: 340L  


## emptyTests

Test: Empty

Solution: 0  
Test case 1. A. Time: 61L

Solution: 2  
Test case 1. A. Time: 62L

Solution: 5  
Test case 1. A. Time: 70L

Input   | Expected        | Result  | Best
---     | ---             | ---     | ---
0       | 0               | 0       | (1, 61)
2       | 2               | 2       | (1, 62)
5       | 5               | 5       | (1, 70)

Averages  
Test case 1. Average Time: 64L

Ranking  
Test case 1. Average Time: 64L

In [ ]:
//// test

inl get_solutions () =
    [
        "A",
        fun n =>
            n + 1f64
    ]

inl rec empty_1_tests () =
    inl test_cases = [
        0, 1
        2, 3
        5, 6
    ]

    inl solutions = get_solutions ()

    // inl is_fast () = true

    inl count =
        if is_fast ()
        then 1000i32
        else 2000000i32

    run_all (nameof empty_1_tests) count solutions test_cases
    |> sort_result_list

empty_1_tests ()



Test: v0

Solution: 0.0  
Test case 1. A. Time: 101  

Solution: 2.0  
Test case 1. A. Time: 63  

Solution: 5.0  
Test case 1. A. Time: 44  

Input	| Expected	| Result	| Best             
---  	| ---     	| ---   	| ---              
0.0  	| 1.0     	| 1.0   	| struct (1L, 101L)
2.0  	| 3.0     	| 3.0   	| struct (1L, 63L) 
5.0  	| 6.0     	| 6.0   	| struct (1L, 44L) 

Average Ranking  
Test case 1. Average Time: 69L  
